In [ ]:
# 🛠️ Install first in Colab if needed
# !pip install gradio inference requests

import gradio as gr
from inference_sdk import InferenceHTTPClient
import requests

# ✅ Roboflow API Setup
CLIENT = InferenceHTTPClient(
    api_url="https://serverless.roboflow.com",
    api_key="zL85GrIxufrU2bfcS2Sj"
)

# ✅ Weather API Key and Endpoint
WEATHER_API_KEY = "64be775708b8774817ae621feb910017"
WEATHER_URL = "https://api.openweathermap.org/data/2.5/weather"

# ✅ Disease Detection Function
def predict_disease(image_path):
    try:
        result = CLIENT.infer(image_path, model_id="plant-disease-detection-v2-2nclk/1")
        if not result["predictions"]:
            return "❌ کوئی بیماری معلوم نہیں ہوئی"
        pred = result["predictions"][0]
        return f"🦠 بیماری: {pred['class']}\nاعتماد: {pred['confidence']*100:.2f}%"
    except Exception as e:
        return f"❌ خرابی: {str(e)}"

# ✅ Weather Fetch Function
def get_weather(city):
    try:
        params = {"q": city, "appid": WEATHER_API_KEY, "units": "metric", "lang": "ur"}
        response = requests.get(WEATHER_URL, params=params)
        data = response.json()

        if data.get("cod") != 200:
            return "❌ موسم کی معلومات حاصل نہیں ہو سکیں"

        weather = data["weather"][0]["description"]
        temp = data["main"]["temp"]
        feels_like = data["main"]["feels_like"]
        humidity = data["main"]["humidity"]

        return (
            f"🌤️ موجودہ موسم: {weather}\n"
            f"🌡️ درجہ حرارت: {temp}°C (محسوس ہوتا ہے جیسے {feels_like}°C)\n"
            f"💧 نمی: {humidity}%"
        )
    except Exception as e:
        return f"❌ موسم کی معلومات میں خرابی: {str(e)}"

# ✅ Combined Function for Gradio
def app_func(image, city):
    disease_result = predict_disease(image)
    weather_result = get_weather(city)
    return disease_result, weather_result

# ✅ Gradio UI
app = gr.Interface(
    fn=app_func,
    inputs=[
        gr.Image(type="filepath", label="📷 پتے کی تصویر اپ لوڈ کریں"),
        gr.Textbox(label="🏙️ شہر کا نام (جیسے Lahore یا Karachi)")
    ],
    outputs=[
        gr.Textbox(label="🔎 بیماری کی پیشگوئی"),
        gr.Textbox(label="🌦️ موسم کی صورتحال")
    ],
    title="🌿 زرعی بیماری اور موسم ایپ",
    description="پتوں کی بیماری اور موسم کی معلومات ایک جگہ پر"
)

app.launch(share=True)
